In [1]:
#------------Import_all_libraries_for_operation-----------------
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
from time import sleep
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
import numpy as np
from sklearn import preprocessing
from time import sleep


In [2]:
#-----------------Read-dataset_in_csv_format------------------------

In [3]:
sentiment_data = pd.read_csv('/home/sampath-pc/Documents/new_sentiment_vin/trainingdata.txt', 
                             sep='\t', names=['Class','Text'])

In [4]:
sentiment_data.head()

,Class,Text
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...


In [5]:
#---------------Get_english_language_stop-words_from_nltk_corpus--------------

stop_word = set(stopwords.words('english'))

#--------------Define_vectorizer_to_remove_stop_words_from_each_sentence----------------
vectorizer =TfidfVectorizer(use_idf=True,lowercase=True
                            ,strip_accents='ascii'
                            ,stop_words=stop_word)

In [6]:
#--------------Extract_target_columns-----------------
y=sentiment_data.Class

In [7]:
#--------------transform_text_to_numeric_dataset---------------
x=vectorizer.fit_transform(sentiment_data.Text)
print (y.shape)
print (x.shape)

(6918,)
(6918, 2011)


In [8]:
#---------Split_dataset_into_train_and_test------------
"""Train_data will be use for training_model while Test_data will be used to 
evaluate model performance"""
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=int(x.shape[0]*0.25),random_state=42)
scaler = preprocessing.StandardScaler(with_mean=False).fit(X_train)
X_train_transformed = scaler.transform(X_train)

print(X_train.shape,X_test.shape)

(5189, 2011) (1729, 2011)


In [9]:
#--------train_model_using_multinomial_naive_bayes---------
clf= naive_bayes.MultinomialNB()   #---------Define_model----------
clf.fit(X_train,Y_train)           #---------Train_model-----------
Y_prediction=clf.predict_proba(X_test) #-----Perform_prediction_using_model----------
print(Y_prediction)     #---------Predict_proba function generate probabilities.

[[1.07399235e-02 9.89260077e-01]
 [4.09960453e-04 9.99590040e-01]
 [4.81570381e-03 9.95184296e-01]
 ...
 [9.97563166e-01 2.43683387e-03]
 [4.97692372e-03 9.95023076e-01]
 [9.88344119e-01 1.16558811e-02]]


In [10]:
Y_pred = Y_prediction[:,1]
classes = []        
for j in Y_pred:
    if j > 0.55:
        classes.append(['Positive class',j])
    elif j < 0.45:
        classes.append(['Negative class',0 - j])
    else:
        classes.append(['Neutral class',j])

print(classes)

[['Positive class', 0.9892600765346681], ['Positive class', 0.9995900395466936], ['Positive class', 0.9951842961908288], ['Positive class', 0.9952079048038668], ['Negative class', -0.0008610901698406418], ['Positive class', 0.9918844911490926], ['Positive class', 0.9993609355289201], ['Negative class', -0.031124998224278146], ['Positive class', 0.9738239259082678], ['Positive class', 0.9750222392142058], ['Negative class', -0.02633675547308563], ['Positive class', 0.9993609355289201], ['Negative class', -0.00022205568158902012], ['Positive class', 0.9987452795348324], ['Negative class', -0.0003514998814759927], ['Positive class', 0.6104514071286966], ['Positive class', 0.9993609355289201], ['Positive class', 0.9957474230952835], ['Positive class', 0.9778558692187214], ['Negative class', -0.0004197551882906944], ['Positive class', 0.9970145028454928], ['Negative class', -0.011655881087167497], ['Positive class', 0.929649867036071], ['Positive class', 0.9750222392142058], ['Positive clas

In [11]:
#-----------capture_classes_and_probabilities_in_dataframe------


class_labels=[]
class_probability=[]
for i in range(0,len(classes)):
    temp=classes[i]
    class_labels.append(temp[0])
    class_probability.append(temp[1])
    
#print(class_labels)
#print(class_probability)

temp1=pd.DataFrame(class_labels,columns=['Class_labels'])
temp2=pd.DataFrame(class_probability,columns=['Class_probabilities'])

df=pd.concat([temp1,temp2],axis=1)
# print(df.head())
print(df['Class_labels'].value_counts())

Positive class    997
Negative class    707
Neutral class      25
Name: Class_labels, dtype: int64


In [12]:
#-------Test_model_performance_model_accuracy-------------
print("Test accuracy is : ")
X_test_transformed = scaler.transform(X_test)
print(clf.score(X_test_transformed, Y_test))

Test accuracy is : 
0.9589358010410642


In [13]:
#---Save_dataframe_to_jason_file------------

In [40]:
df.to_json("Sentiment_results_1.JSON")

In [41]:
""" As probability value never be negative. It always lies between (0,1), but for the purpos the probabilities for
negative classes should be lie in between (-1,0). To achive this I 
made slight adjustment by defining some threshold value. In general practice we use threshold value
to generate class label decision from probabilities."""

' As probability value never be negative. It always lies between (0,1), but for the purpos the probabilities for\nnegative classes should be lie in between (-1,0). To achive this I \nmade slight adjustment by defining some threshold value. In general practice we use threshold value\nto generate class label decision from probabilities.'

In [42]:
#----------------Implement_above_model_on_test_reviews_dataset-----------------

In [43]:
import json
with open('Test_text.JSON') as f:
    data = json.load(f)

In [44]:
text=pd.DataFrame(data)
print(text)

                                            Text_set
0  Fed's Charles Plosser sees high bar for change...
1  Policy shouldn’t be “whip-sawed from month to ...
2  The numbers were “encouraging,” though they fo...
3  The Fed has pledged to keep the main interest ...
4  Plosser, a former University of Rochester econ...
5  Fed's Plosser: Nasty Weather Has Curbed Job Gr...
6  In a speech at the Bank of France, Philadelphi...


In [45]:
Y_pred=[]
for i in range(0,len(text)):
    #---------Implement_trained_model_on_test_review_text--------
    text_numeric=vectorizer.transform(text.iloc[i])
    Y_prediction=clf.predict_proba(text_numeric)
    Y_pred.append(Y_prediction[:,1])

In [46]:
Y_pred

[array([0.75538688]),
 array([0.86382681]),
 array([0.74378002]),
 array([0.14010586]),
 array([0.56870303]),
 array([0.64742644]),
 array([0.44975557])]

In [47]:
classes = []        
for j in Y_pred:
    if j > 0.55:
        classes.append(['Positive class',j])
    elif j < 0.45:
        classes.append(['Negative class',0 - j])
    else:
        classes.append(['Neutral class',j])

print(classes)

[['Positive class', array([0.75538688])], ['Positive class', array([0.86382681])], ['Positive class', array([0.74378002])], ['Negative class', array([-0.14010586])], ['Positive class', array([0.56870303])], ['Positive class', array([0.64742644])], ['Negative class', array([-0.44975557])]]


In [48]:
#-----------capture_classes_and_probabilities_in_dataframe------
class_labels=[]
class_probability=[]
for i in range(0,len(classes)):
    temp=classes[i]
    class_labels.append(temp[0])
    class_probability.append(temp[1])
    
#print(class_labels)
#print(class_probability)

temp1=pd.DataFrame(class_labels,columns=['Class_labels'])
temp2=pd.DataFrame(class_probability,columns=['Class_probabilities'])

In [49]:
final_df=pd.concat([temp1,temp2],axis=1)
final_df.head()

,Class_labels,Class_probabilities
0,Positive class,0.755387
1,Positive class,0.863827
2,Positive class,0.743780
3,Negative class,-0.140106
4,Positive class,0.568703
